# 2NF
Second Normal Form (2NF) is a level of database normalization that aims to reduce data redundancy and ensure data integrity. It builds upon the First Normal Form (1NF) and introduces the concept of fully functional dependency. Below is a detailed explanation:

### Definition of 2NF
A database table is in **Second Normal Form (2NF)** if:
1. It is in **First Normal Form (1NF)**, meaning:
   - The table has no repeating groups or arrays, i.e., each column contains atomic values.
   - Each row is unique, identified by a primary key.
2. It has **no partial dependency**, meaning:
   - **Non-prime attributes** (attributes not part of any candidate key) must depend on the **entire primary key** and not just a part of it.



### Example: Violating 2NF
Consider a table storing information about students and courses they enroll in:
| StudentID | CourseID | StudentName | CourseName |
|-----------|----------|-------------|------------|
| S01       | C01      | Alice       | Math       |
| S02       | C02      | Bob         | Physics    |

Here:
- **Primary Key**: (StudentID, CourseID) (composite key).
- **Partial Dependency**:
  - `StudentName` depends only on `StudentID`.
  - `CourseName` depends only on `CourseID`.

This violates 2NF because `StudentName` and `CourseName` do not depend on the entire composite key.

### Converting to 2NF
To resolve this, split the table into smaller tables to remove partial dependency:

1. **Student Table**:
   | StudentID | StudentName |
   |-----------|-------------|
   | S01       | Alice       |
   | S02       | Bob         |

2. **Course Table**:
   | CourseID | CourseName |
   |----------|------------|
   | C01      | Math       |
   | C02      | Physics    |

3. **Enrollment Table**:
   | StudentID | CourseID |
   |-----------|----------|
   | S01       | C01      |
   | S02       | C02      |

Now, all non-prime attributes fully depend on the primary keys of their respective tables.

### Advantages of 2NF
- Eliminates partial dependencies, reducing redundancy.
- Makes the database structure more logical and organized.
- Improves data integrity and consistency.

### Limitations of 2NF
- Does not address **transitive dependency** (covered in 3NF).
- Might lead to more joins during query execution.


# Checkpoints
**Checkpoints** in database systems are mechanisms used to enhance the efficiency of database recovery in case of a failure. They act as predefined points of consistency within the database transaction log, marking moments where all transactions and data changes have been committed to the physical database storage.

### Purpose of Checkpoints
- **Facilitate Recovery**: Reduce the amount of work during recovery by avoiding re-processing of the entire transaction log.
- **Improve Performance**: By saving consistent states of the database, checkpoints minimize the time needed to restore the database after a crash.
- **Limit Log Size**: By writing committed changes to disk at checkpoints, it prevents the transaction log from growing indefinitely.

### How Checkpoints Work
1. **Log Writing**: The database system maintains a transaction log that records all operations performed, including changes made to the data and metadata.
2. **Checkpoint Creation**: At regular intervals or predefined triggers, the database system:
   - Flushes all in-memory buffers to disk, ensuring that data pages are up-to-date.
   - Writes a checkpoint record in the transaction log, indicating that all transactions up to that point are committed and persistent.
3. **Recovery Assistance**: During recovery, the database uses the most recent checkpoint to begin log replay, skipping all previous transactions that are already safely stored.

### Types of Checkpoints
1. **Manual Checkpoints**: Triggered by the database administrator using commands.
2. **Automatic Checkpoints**: Triggered by the database system based on time intervals, transaction volume, or resource utilization.


### Benefits of Checkpoints
1. **Faster Recovery**: Reduces the amount of log records to process during recovery.
2. **Data Integrity**: Ensures a consistent state of the database is periodically recorded.
3. **Efficient Resource Use**: Frees up memory buffers by committing changes to the disk.

### Challenges
1. **Performance Overhead**: Writing a large amount of data to disk during checkpointing can cause temporary performance degradation.
2. **Granularity**: Deciding the frequency of checkpoints is critical; too frequent checkpoints may slow down the system, while infrequent ones increase recovery time.

### Practical Use in Recovery
- **Crash Recovery**: If a database crashes, the recovery process:
  1. Finds the most recent checkpoint in the transaction log.
  2. Restores the database state from the checkpoint.
  3. Replays only the transactions recorded after the checkpoint.
- **Undo/Redo Operations**:
  - **Undo**: Rollback uncommitted transactions that were active during the crash.
  - **Redo**: Reapply committed transactions after the checkpoint.

Here’s an example illustrating **Checkpoints** in the format `<Tn, Action>` as used in a transaction log. This example shows how checkpoints assist in recovery:

---

### **Scenario**  
A database processes three transactions: **T1**, **T2**, and **T3**. During the process, a checkpoint is created. Later, a crash occurs.

---

### **Transaction Log Entries**

| Log Entry          | Description                                     |
|--------------------|-------------------------------------------------|
| `<T1, Start>`      | Transaction **T1** starts.                     |
| `<T1, Write(A)>`   | Transaction **T1** updates data item **A**.     |
| `<T2, Start>`      | Transaction **T2** starts.                     |
| `<T1, Commit>`     | Transaction **T1** commits.                    |
| `<T2, Write(B)>`   | Transaction **T2** updates data item **B**.     |
| `<T3, Start>`      | Transaction **T3** starts.                     |
| `<Checkpoint>`     | A checkpoint is created.                       |
| `<T2, Write(C)>`   | Transaction **T2** updates data item **C**.     |
| `<T3, Write(D)>`   | Transaction **T3** updates data item **D**.     |
| `<T2, Commit>`     | Transaction **T2** commits.                    |
| **Crash Occurs**   | The system crashes at this point.              |

---

### **Recovery Process**
1. **Locate the Last Checkpoint**:
   - The last checkpoint is `<Checkpoint>`. Transactions before the checkpoint (`T1`) are already committed or aborted and do not require recovery.

2. **Redo Transactions After Checkpoint**:
   - Replay committed transactions **T2** to apply their updates to the database:
     - `<T2, Write(C)>`
     - `<T2, Commit>`.

3. **Undo Unfinished Transactions**:
   - Undo updates made by **T3** because it was not committed:
     - `<T3, Write(D)>` is undone.

---

### **Final Actions**
- The database is restored to a consistent state:
  - Updates from **T1** and **T2** are retained (as they were committed).
  - Partial updates from **T3** are rolled back.

---

### **Simplified Log Post-Recovery**

| Log Entry          | Description                                     |
|--------------------|-------------------------------------------------|
| `<T1, Start>`      | (No recovery needed, committed before checkpoint). |
| `<T2, Start>`      | Re-applied updates from **T2**.                |
| `<Checkpoint>`     | Used as the recovery starting point.           |
| `<T3, Start>`      | Changes undone for this uncommitted transaction.|

---

This format shows how checkpoints reduce the work required during recovery by limiting redo/undo operations to transactions after the checkpoint.

# Data Dependency and Domain Dependency
### **1. Domain Dependency**
**Domain Dependency** refers to the constraints or rules that define the valid set of values a database column (attribute) can take. It is based on the **domain** of an attribute, which is the permissible range or set of values for that attribute.

#### Characteristics of Domain Dependency:
- Ensures **data integrity** by restricting values in a column to the defined domain.
- A domain can be defined explicitly (e.g., a range of numbers) or implicitly (e.g., a reference table).
- Examples include:
  - Data type constraints (e.g., `integer`, `varchar`).
  - Value range constraints (e.g., age must be between `0` and `120`).
  - Pattern constraints (e.g., email should match a regex pattern).

#### Example:
- A table `Employee` has a column `Age` with a domain constraint: `Age` must be an integer between `18` and `65`.
  
| EmployeeID | Name       | Age |
|------------|------------|-----|
| 1          | Alice      | 25  |
| 2          | Bob        | 17  |  *(Violates domain dependency as age < 18)*

In this case, inserting or updating the row with `Age = 17` would violate the **domain dependency**.

---

### **2. Data Dependency**
**Data Dependency** refers to the relationship or dependency between attributes in a database. It focuses on how one attribute's value is dependent on another attribute's value, based on the database's functional or logical constraints.

#### Types of Data Dependencies:
1. **Functional Dependency**:
   - Attribute `B` is functionally dependent on attribute `A` if the value of `A` determines the value of `B` (denoted as `A → B`).
   - Example: In a `Student` table, `RollNumber → Name` because each roll number uniquely identifies a student's name.

2. **Transitive Dependency**:
   - An attribute is indirectly dependent on another attribute through a third attribute.
   - Example: `A → B` and `B → C` implies `A → C`.

3. **Partial Dependency**:
   - In a table with a composite primary key, an attribute depends on only a part of the composite key.
   - Example: If `(A, B)` is the primary key, and `C` depends only on `A`, it creates a partial dependency.

4. **Multivalued Dependency**:
   - Occurs when one attribute in a table determines multiple values of another attribute independently of other attributes.
   - Example: A student can be enrolled in multiple courses, and the dependency is independent of their address.

#### Example:
Consider a table `Orders`:
| OrderID | CustomerID | CustomerName | TotalAmount |
|---------|------------|--------------|-------------|
| 1       | C101       | Alice        | 500         |
| 2       | C102       | Bob          | 300         |

- **Functional Dependency**:
  - `CustomerID → CustomerName` (CustomerID determines CustomerName).
  - `OrderID → (CustomerID, TotalAmount)` (OrderID determines the other attributes).

---

### Key Differences
| Aspect                 | Domain Dependency                         | Data Dependency                                  |
|------------------------|-------------------------------------------|------------------------------------------------|
| **Focus**              | Constraints on attribute values.          | Relationship between attributes.               |
| **Scope**              | Attribute-level validation.               | Intra-record or inter-record relationships.    |
| **Examples**           | Data types, ranges, patterns.             | Functional, partial, and transitive dependencies. |
| **Purpose**            | Ensures valid and meaningful data input.  | Ensures logical consistency and normalization. | 

Both are crucial for maintaining **data integrity** and ensuring the database's design adheres to the defined rules and relationships.

# Finding Candidate Key  

Consider a relation $ R(A, B, C, D, E, G) $ with the following set of functional dependencies:  
1. $ AB \rightarrow C $  
2. $ D \rightarrow EG $  
3. $ C \rightarrow A $  
4. $ BE \rightarrow C $  
5. $ BC \rightarrow D $  
6. $ CG \rightarrow BD $  
7. $ ACD \rightarrow B $  
8. $ CE \rightarrow AG $  

Prove that $ BD $ is a candidate key for the relation $ R $.  

To prove that **BD** is a candidate key for the given set of functional dependencies, we must show two things:

1. **BD can determine all attributes in the relation** (uniqueness property).
2. **No proper subset of BD can determine all attributes** (minimality property).

---

### **Step 1: Attributes in the Relation**
From the given functional dependencies, the attributes in the relation are:
- $ A, B, C, D, E, G $.

---

### **Step 2: Given Functional Dependencies**
1. $ AB \rightarrow C $
2. $ D \rightarrow EG $
3. $ C \rightarrow A $
4. $ BE \rightarrow C $
5. $ BC \rightarrow D $
6. $ CG \rightarrow BD $
7. $ ACD \rightarrow B $
8. $ CE \rightarrow AG $

---

### **Step 3: Verify if BD is a Superkey**
To prove $ BD $ is a superkey, compute the **closure** of $ BD $ ($ BD^+ $) and check if it includes all attributes ($ A, B, C, D, E, G $).

#### Compute $ BD^+ $ Step-by-Step:
1. Start with $ BD $:
   $$
   BD^+ = \{ B, D \}
   $$

2. From $ BC \rightarrow D $:
   - Since $ BD $ already includes $ B $ and $ D $, $ BC \rightarrow D $ does not add any new attributes.

3. From $ CG \rightarrow BD $:
   - $ BD $ already includes $ B, D $, so $ CG $ does not add new attributes yet.

4. From $ D \rightarrow EG $:
   - $ BD $ includes $ D $, so we can add $ E, G $ to $ BD^+ $:
     $$
     BD^+ = \{ B, D, E, G \}
     $$

5. From $ BE \rightarrow C $:
   - $ BD^+ $ includes $ B $ and $ E $, so we can add $ C $ to $ BD^+ $:
     $$
     BD^+ = \{ B, D, E, G, C \}
     $$

6. From $ C \rightarrow A $:
   - $ BD^+ $ includes $ C $, so we can add $ A $ to $ BD^+ $:
     $$
     BD^+ = \{ A, B, C, D, E, G \}
     $$

Now $ BD^+ $ includes all attributes ($ A, B, C, D, E, G $), so $ BD $ is a **superkey**.

---

### **Step 4: Verify if BD is Minimal**
To prove $ BD $ is a **candidate key**, we need to ensure no proper subset of $ BD $ is a superkey.

#### Test $ B^+ $:
Start with $ B $:
$$
B^+ = \{ B \}
$$
- $ B $ does not determine $ D $, so $ B $ alone cannot be a superkey.

#### Test $ D^+ $:
Start with $ D $:
$$
D^+ = \{ D \}
$$
- From $ D \rightarrow EG $, $ D^+ = \{ D, E, G \} $.
- $ D $ does not determine $ A, B, C $, so $ D $ alone cannot be a superkey.

Since neither $ B $ nor $ D $ alone is a superkey, $ BD $ is **minimal**.

---

### **Conclusion**
1. $ BD^+ $ includes all attributes, so $ BD $ is a superkey.
2. No proper subset of $ BD $ is a superkey, so $ BD $ is minimal.

Therefore, $ BD $ is a **candidate key** for the relation.

# Check Level of Normalization
To determine whether the given relation $ \text{RENT\_DUE} $ is in **2NF**, **3NF**, or **BCNF**, we need to analyze the functional dependencies (FDs) and the keys of the relation step by step.

---

### **Step 1: Attributes of the Relation**
The relation $ \text{RENT\_DUE} $ has the following attributes:
$$
\{ \text{qtr\_no}, \text{emp\_id}, \text{date\_of\_occupancy}, \text{rent} \}.
$$

---

### **Step 2: Functional Dependencies (FDs)**
The given FDs are:
1. $ \text{qtr\_no, date\_of\_occupancy} \rightarrow \text{emp\_id, rent} $
2. $ \text{emp\_id} \rightarrow \text{qtr\_no} $

---

### **Step 3: Candidate Key**
A candidate key is an attribute or a set of attributes that uniquely identifies each tuple in the relation.  
- From FD 1: $ \text{qtr\_no, date\_of\_occupancy} \rightarrow \text{emp\_id, rent} $, the combination of $ \text{qtr\_no, date\_of\_occupancy} $ determines all other attributes, so it is a **candidate key**.  
- FD 2 does not provide any new keys, as $ \text{emp\_id} $ does not determine all attributes.

Thus, the **candidate key** is:
$$
\{\text{qtr\_no, date\_of\_occupancy}\}.
$$

---

### **Step 4: Normal Forms Analysis**

#### **A. First Normal Form (1NF)**
The relation is in **1NF** because all attributes contain atomic (indivisible) values.

---

#### **B. Second Normal Form (2NF)**

A relation is in **2NF** if:
1. It is in **1NF**.
2. **No partial dependency** exists, meaning no non-prime attribute is dependent on a part of the candidate key.

Here:
- The candidate key is $ \{\text{qtr\_no, date\_of\_occupancy}\} $.
- Non-prime attributes are $ \{\text{emp\_id}, \text{rent}\} $.

From the FDs:
- $ \text{qtr\_no, date\_of\_occupancy} \rightarrow \text{emp\_id, rent} $: No partial dependency here.
- $ \text{emp\_id} \rightarrow \text{qtr\_no} $: This FD does not involve a subset of the candidate key.

Since there are no partial dependencies, the relation is in **2NF**.

---

#### **C. Third Normal Form (3NF)**

A relation is in **3NF** if:
1. It is in **2NF**.
2. For every FD $ X \rightarrow Y $, at least one of the following holds:
   - $ X $ is a superkey.
   - $ Y $ is a prime attribute (part of some candidate key).

Analyze the FDs:
1. $ \text{qtr\_no, date\_of\_occupancy} \rightarrow \text{emp\_id, rent} $:  
   - $ \{\text{qtr\_no, date\_of\_occupancy}\} $ is a superkey, so this satisfies 3NF.

2. $ \text{emp\_id} \rightarrow \text{qtr\_no} $:  
   - $ \text{emp\_id} $ is not a superkey.
   - $ \text{qtr\_no} $ is not a prime attribute.

Thus, the second FD violates **3NF**.

---

#### **D. Boyce-Codd Normal Form (BCNF)**

A relation is in **BCNF** if:
1. It is in **3NF**.
2. For every FD $ X \rightarrow Y $, $ X $ is a superkey.

Analyze the FDs:
1. $ \text{qtr\_no, date\_of\_occupancy} \rightarrow \text{emp\_id, rent} $:  
   - $ \{\text{qtr\_no, date\_of\_occupancy}\} $ is a superkey, so it satisfies BCNF.

2. $ \text{emp\_id} \rightarrow \text{qtr\_no} $:  
   - $ \text{emp\_id} $ is not a superkey.

Thus, the second FD violates **BCNF**.

---

### **Step 5: Conclusion**

- The relation is in **2NF** because it has no partial dependencies.
- The relation is **not in 3NF** because $ \text{emp\_id} \rightarrow \text{qtr\_no} $ violates the conditions of 3NF.
- The relation is **not in BCNF** because $ \text{emp\_id} $ is not a superkey in $ \text{emp\_id} \rightarrow \text{qtr\_no} $.

To achieve BCNF, the relation must be decomposed into two relations:
1. $ R1(\text{emp\_id}, \text{qtr\_no}) $ with FD $ \text{emp\_id} \rightarrow \text{qtr\_no} $.
2. $ R2(\text{qtr\_no}, \text{date\_of\_occupancy}, \text{rent}) $ with FD $ \text{qtr\_no, date\_of\_occupancy} \rightarrow \text{rent} $.